<a href="https://colab.research.google.com/github/skrmanglam/MNIST/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

Load MNIST

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Normalize the data
Build training pipeline

Apply the following transormations:

    ds.map: TFDS provide the images as tf.uint8, while the model expect tf.float32, so normalize images
    ds.cache As the dataset fit in memory, cache before shuffling for better performance.
    Note: Random transformations should be applied after caching
    ds.shuffle: For true randomness, set the shuffle buffer to the full dataset size.
    Note: For bigger datasets which do not fit in memory, a standard value is 1000 if your system allows it.
    ds.batch: Batch after shuffling to get unique batches at each epoch.
    ds.prefetch: Good practice to end the pipeline by prefetching for performances.


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

Build evaluation pipeline

Testing pipeline is similar to the training pipeline, with small differences:

    No ds.shuffle() call
    Caching is done after batching (as batches can be the same between epoch)


In [ ]:
ds_test=ds_test.map( normalize_img,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.cache()
ds_test=ds_test.prefetch(tf.data.experimental.AUTOTUNE)

Create and train the model

In [ ]:
model= tf.keras.models.Sequential([
                                   tf.keras.layers.Flatten(input_shape=(28,28)),
                                   tf.keras.layers.Dense(128,activation='relu'),
                                   tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
tf
model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 9s 7ms/step - loss: 0.3594 - sparse_categorical_accuracy: 0.9014 - val_loss: 0.1988 - val_sparse_categorical_accuracy: 0.9427
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1647 - sparse_categorical_accuracy: 0.9529 - val_loss: 0.1402 - val_sparse_categorical_accuracy: 0.9594
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1171 - sparse_categorical_accuracy: 0.9668 - val_loss: 0.1062 - val_sparse_categorical_accuracy: 0.9679
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0899 - sparse_categorical_accuracy: 0.9747 - val_loss: 0.0981 - val_sparse_categorical_accuracy: 0.9715
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0724 - sparse_categorical_accuracy: 0.9793 - val_loss: 0.0880 - val_sparse_categorical_accuracy: 0.9722
Epoch 6/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0598 - sparse_categorical_accu